In [1]:
import numpy as np
import sklearn.metrics as metrics
import pickle

IF=open("assignment3_천억.txt",'r')
lst_code_date=[]
trainX=[]
trainY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    trainX.append([])
    tmpx = list(map(int, x.split(",")))
    for i in range(0,len(tmpx),5):
        trainX[-1].append(tmpx[i:i+5])
    trainY.append(int(y))
trainX=np.array(trainX)
trainY=np.array(trainY)

In [37]:
line

'028080\t20210629n\tsell\tall\n'

In [17]:
import os
import time
import pandas as pd
import numpy as np
import datetime
import pymysql
from sqlalchemy import create_engine
import FinanceDataReader as fdr
from tqdm import tqdm

db_dsml = pymysql.connect(
    host = 'localhost', 
    port = 3306, 
    user = 'stock_user', 
    passwd = 'bigdata', 
    db = 'stock', 
    charset = 'utf8'
)

cursor = db_dsml.cursor()

In [19]:
db_dsml = pymysql.connect(
    host = 'localhost', 
    port = 3306, 
    user = 'stock_user', 
    passwd = 'bigdata', 
    db = 'stock', 
    charset = 'utf8'
)

cursor = db_dsml.cursor()

In [2]:
trainX.shape

(14120, 10, 5)

In [3]:
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(trainX.shape[1],trainX.shape[2])))
model.add(keras.layers.LSTM(128, activation='hard_sigmoid', return_sequences=True))
model.add(keras.layers.LSTM(64, activation='hard_sigmoid', return_sequences=True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(64, activation='hard_sigmoid', return_sequences=True))
model.add(keras.layers.LSTM(32, activation='hard_sigmoid', return_sequences=False))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))
    
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(0.01,decay_steps=100000,decay_rate=0.96)), 
    loss="binary_crossentropy",
    metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 128)           68608     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 64)            49408     
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 64)            33024     
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 3

2022-01-25 13:49:25.177253: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-25 13:49:25.177397: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-25 13:49:25.183096: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
es = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=20, verbose=1, mode="auto", restore_best_weights=True)
lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
sv = keras.callbacks.ModelCheckpoint(
    "./model_keras_lstm.h5", monitor='val_accuracy', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
    )

history = model.fit(trainX, trainY, validation_split=0.20, batch_size=128, epochs=50, callbacks=[es, sv], shuffle=True)            

2022-01-25 13:49:35.850887: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-25 13:49:35.867187: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2894595000 Hz


Epoch 1/50
89/89 [==============================] - 15s 115ms/step - loss: 0.5127 - accuracy: 0.7821 - val_loss: 0.4917 - val_accuracy: 0.8106

Epoch 00001: val_accuracy improved from -inf to 0.81055, saving model to ./model_keras_lstm.h5
Epoch 2/50
89/89 [==============================] - 13s 141ms/step - loss: 0.4659 - accuracy: 0.8266 - val_loss: 0.4863 - val_accuracy: 0.8106

Epoch 00002: val_accuracy did not improve from 0.81055
Epoch 3/50
89/89 [==============================] - 11s 126ms/step - loss: 0.4802 - accuracy: 0.8175 - val_loss: 0.4870 - val_accuracy: 0.8106

Epoch 00003: val_accuracy did not improve from 0.81055
Epoch 4/50
89/89 [==============================] - 12s 130ms/step - loss: 0.4835 - accuracy: 0.8134 - val_loss: 0.4857 - val_accuracy: 0.8106

Epoch 00004: val_accuracy did not improve from 0.81055
Epoch 5/50
89/89 [==============================] - 14s 162ms/step - loss: 0.4721 - accuracy: 0.8208 - val_loss: 0.4855 - val_accuracy: 0.8106

Epoch 00005: val_acc

In [6]:
import numpy as np
import sklearn.metrics as metrics
import pickle
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

IF=open("assignment3-2_천억.txt",'r')
lst_code_date=[]
testX=[]
testY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    testX.append([])
    tmpx = list(map(int, x.split(",")))
    for i in range(0,len(tmpx),5):
        testX[-1].append(tmpx[i:i+5])    
    testY.append(int(y))
testX=np.array(testX)
testY=np.array(testY)


In [12]:
# load a saved model
import tensorflow as tf
tf.keras.models.load_model('./model_keras_lstm.h5')

predY = model.predict(testX)
print(predY)

# saved_model = load_model('./model_keras_lstm.h5')


[[0.14827934]
 [0.1482805 ]
 [0.14828047]
 ...
 [0.14828047]
 [0.14828047]
 [0.14828047]]


In [31]:
for i in predY:
    print(i)

[0.14827934]
[0.1482805]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]


[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]

[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828071]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]
[0.14828047]

In [32]:
lst_output=[]
for (code, date), y in zip(lst_code_date, predY):
    if y[0] >= 0.148:
        lst_output.append([code, date, "buy", "r20"])
        lst_output.append([code, date+"n", "sell", "all"])

lst_output.sort(key=lambda x:x[1])

OF=open("assignment4.txt",'w')
for row in lst_output:
    OF.write("\t".join(map(str, row))+"\n")
OF.close()

In [33]:
start_money = 10000000 # 초기 현금 1천만원
money = start_money
dic_code2num ={}  # 보유 종목

IF=open("assignment4--.txt",'r')


for i, line in enumerate(tqdm(IF)): #주문 일지를 한 줄 읽어 옴
    code, date, request, amount = line.strip().split("\t")
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                between  '2021-01-01' AND '2021-08-01'
                '''.format(code,date)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock1 = stock.values.tolist()
        
    
    if 'n' in date:
        for ii, row in enumerate(lst_stock1):
            date2 = row[0].strftime("%Y%m%d")
            if date2== date.rstrip('n'):
                nextday_close=lst_stock1[ii+1][4]
            
    else:
        for row2 in lst_stock1:
            today_date=row2[0].strftime("%Y%m%d")
            if today_date in date:
                today_close= row2[4]

                
                
    if request == 'buy': # buy인 경우
        if amount.startswith('r'):
            request_money = money * float(amount.lstrip("r")) / 100
        elif amount == 'all':
            request_money = money
        elif amount.isdigit():
            request_money = int(amount)
            
        # elif amount == ~~~~~    ##### 기타 필요한 매수 요청 옵션이 있을 시 작성
        else:
            raise Exception('Not permitted option')
        request_money = min(request_money, money)
        buy_num = int(request_money / today_close)
        money -= buy_num * today_close  # 현재 금액(money)을 실제 매수액을 뺀 만큼 업데이트
        if code not in dic_code2num:
            dic_code2num[code] = 0
        dic_code2num[code] += buy_num # 보유 종목 데이터에 구매 종목(code)를 매수 개수 만큼 증가
    if request == 'sell': # sell인 경우
        if amount == 'all':
            sell_num = dic_code2num[code]
        # elif amount == ~~~~~    ##### 기타 필요한 매도 요청 옵션이 있을 시 작성
        else:
            raise Exception('Not permitted option')            
        money += sell_num * nextday_close
        dic_code2num[code] -= sell_num
        if dic_code2num[code] == 0:
            del dic_code2num[code]
IF.close()            
            
if dic_code2num != {}: # 매매가 종료되었는데 보유 종목이 있으면
    raise Exception('Not empty stock') 

print("Final earning rate : {:.2f} %".format(float((money-start_money) / start_money * 100)))

9810it [00:31, 314.24it/s]

Final earning rate : 15.37 %
